## Context managers

Context managers are useful when you want to create an object that automatically initializes itself before executing a code block and cleans up itself after executing the code.

In short, context managers help with managing global state before and after a block of code executes.

First, here is an example of writing text to a file using `try` and `finally` manually:

In [ ]:

file = open('new_file.txt', mode='wt')
try:
    file.write('data goes here')
finally:
    file.close()

In this code, `finally` makes sure to close the open file, even in a case an exception is raised. The `with` statement offers a more elegant way of expressing this:

In [ ]:
with open('new_file.txt', 'wt) as f:
    f.write('data')

The `with` statement handles opening the file and returning the handle, but most importantly, if there is an error when you're writing this file the context handles the exception and makes sure the file is saved properly, and data is written to the file as expected.

A context manager is an object to be used in a `with` statement. A file object is one common example of a context manager. All context managers creates their own contexts using methods called `__enter__` and `__exit__`:

In [34]:
with open('new_file.txt', mode='rt') as file:
    print(dir(file))

['_CHUNK_SIZE', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__iter__', '__le__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '_checkClosed', '_checkReadable', '_checkSeekable', '_checkWritable', 'buffer', 'close', 'closed', 'detach', 'encoding', 'errors', 'fileno', 'flush', 'isatty', 'line_buffering', 'mode', 'name', 'newlines', 'read', 'readable', 'readline', 'readlines', 'seek', 'seekable', 'tell', 'truncate', 'writable', 'write', 'writelines']


When a file is opened using the ``with`` statement, the file object invokes its ``__enter__`` method. After executing the code within ``with`` statement, the file object invokes the ``__exit__`` method, which closes the open file.


To check these method invocations, let's create our own simple context manager.

In [35]:
class CM(object):
    def do(self):
        print("I love Python.")
        
    def __enter__(self):
        print("__enter__")
        return self
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("__exit__")

In [36]:
with CM() as obj:
    obj.do()

__enter__
I love Python.
__exit__


As you can see, when a context manager object is created with the `with` statement, `__enter__` and `__exit__` methods are automatically called to execute a context for the object.

## Easily creating context managers

The same pattern applies when using other context managers -- code is executed when the block is opened, potentially providing an object, and the executes code when the block closes. It is easy to write your own context manager that follows this pattern. You will use the `contextlib.contextmanager` decorator around a function which will `yield` a value. Everything in front of the `yield` will be executed before the start of the block (in `__enter__`); everything afterwards will execute at the close of the block (in `__exit__`).

Consider working with Pandas DataFrames: it is sometimes useful to be able to see every single value in the DataFrame all at once. It is easy to set the display limits of DataFrames by setting `max_rows = None` and `max_columns = None` from `pandas.options.display`, but it's likely you'll only want to do this occasionally. So you can make a context manager that will handle the display, and reset it back to the original value on exit.

First, read some data, and set some usefully small values for `max_rows` and `max_columns` to provide snapshots of the data:

In [4]:
import pandas as pd

pd.options.display.max_rows = 5
pd.options.display.max_columns = 5

olympics = pd.read_csv('Data/olympics2012.csv')
olympics

,Country,Gold,Silver,Bronze
0,Afghanistan,0,0,1
1,Albania,0,0,0
...,...,...,...,...
202,Zambia,0,0,0
203,Zimbabwe,0,0,0


Then write the context manager as follows:

In [13]:
from contextlib import contextmanager

@contextmanager
def display_full_dataframe():
    # This code is run before the context block:
    old_max_rows = pd.options.display.max_rows
    old_max_columns = pd.options.display.max_columns
    pd.options.display.max_rows = None
    pd.options.display.max_columns = None

    yield

    # This code is run after the context block:
    pd.options.display.max_rows = old_max_rows
    pd.options.display.max_columns = old_max_columns

When working from within Jupyter notebook, we'd need to change the following setting so that the DataFrame's is displayed even from within the context:

In [25]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'last'

Now:

In [29]:
with display_full_dataframe():
    olympics[:10]

,Country,Gold,Silver,Bronze
0,Afghanistan,0,0,1
1,Albania,0,0,0
2,Algeria,1,0,0
3,American Virgin Islands,0,0,0
4,Andorra,0,0,0
5,Angola,0,0,0
6,Antigua and Barbuda,0,0,0
7,Argentina,1,1,2
8,Armenia,0,1,2
9,Aruba,0,0,0


After with `with` block, the 5-row limit is reinstated:

In [30]:
olympics[:10]

,Country,Gold,Silver,Bronze
0,Afghanistan,0,0,1
1,Albania,0,0,0
...,...,...,...,...
8,Armenia,0,1,2
9,Aruba,0,0,0


### Exercise: Write a context manager to temporarily change the working directory

Following the pattern above write your own context manager called `move_directory` to change directory for the duration of the with statement. Look at `os.chdir` and `os.getcwd` for changing the Python process' working environment.

Test this with:

```python
with move_directory('Data'):
    forbes = pd.read_csv('forbes1964.csv')
    with display_full_dataframe():
        forbes
```